In [1]:
from selenium import webdriver as wd
from time import sleep
import pandas as pd
import re
from bs4 import BeautifulSoup
import math

In [2]:
#데이터 프레임 형성
df = pd.DataFrame(columns={'name','location','tags'})

In [3]:
tour_spot="3f36ca4b-6f45-45cb-9042-265c96a4868c"
culture_spot="651c5b95-a5b3-11e8-8165-020027310001"
reports_spot="e6875575-2cc2-43ba-9651-28d31a7b3e23"
shopping_spot="0f29b431-75ac-4ab4-a892-b247d516b31d"
indoor_spot="2d4f4e06-2d37-4e54-ad5c-172add6e6680"
activity_spot="23bc02b8-da01-41bf-8118-af882436cd3c"
historical_spot="d3fd4d9f-fbd4-430f-b5d5-291b4d9920be"
nature_spot="c24d515f-3202-45e5-834e-1a091901aeff"

spot_ls=[tour_spot,culture_spot,reports_spot,shopping_spot,indoor_spot,activity_spot,historical_spot,nature_spot]

In [4]:
def clickTags(page, loc_id):
    button=page.find_element_by_id(loc_id)
    button.click()

In [5]:
def getLastPageIndex(page):
    totalCnt = page.find_element_by_id('totalCnt').text
    print(totalCnt)
    lastIndex = math.ceil(int(totalCnt.replace(',',''))/10)
    return lastIndex

In [6]:
def clickNextPage(page, index):
    try: 
        if (index%5 == 0):
            button=page.find_element_by_css_selector('a.btn_next')
            button.click()
        else:
            page_box=page.find_element_by_css_selector('#contents > div.wrap_contView.clfix > div.box_leftType1 > div.page_box')
            button=page_box.find_element_by_id((str(index+1)))
            button.click()
    except Exception as ex:
        print(ex)

In [7]:
def eachPageInfo(page):
    info=page.find_elements_by_css_selector("li.bdr_nor")
    df = pd.DataFrame(columns={'name','location','tags'})
    for i in info:
        name= i.find_element_by_css_selector("div.tit").text
        location= i.find_element_by_css_selector("p").text.split('서울 ')[1]
        tag= i.find_element_by_css_selector("p.tag").text
        df= df.append({'name': name, 'location': location, 'tags': tag}, ignore_index=True)
        
    return df

In [33]:
def main():
    #데이터 프레임 형성
    df = pd.DataFrame(columns={'name','location','tags'})
    #셀레니엄으로 크롬 열기
    url = "https://korean.visitkorea.or.kr/list/ms_list.do?areacode=1"
    driver = wd.Chrome()
    driver.get(url)
    sleep(3)
    try:
        for i in spot_ls:
            clickTags(driver, i)
            #pagination
            sleep(3)
            last_page = getLastPageIndex(driver)
            for j in range(1,last_page+1):
                sleep(1)
                df_new = eachPageInfo(driver)
                df_new['category']=int(spot_ls.index(i)+1)
                df = pd.concat([df, df_new], ignore_index=True)
                if(j < last_page):
                    clickNextPage(driver, j)
            clickTags(driver, i)
            sleep(5)
        return df
    except Exception as ex:
        print(ex)
        return df

In [34]:
result= main()

699
423
116
278
93
Message: element click intercepted: Element <a href="javascript:" id="...">5</a> is not clickable at point (464, 694). Other element would receive the click: <div class="loading" style="z-index: 100000; display: block;">...</div>
  (Session info: chrome=80.0.3987.163)

65
218
50


In [37]:
result

,tags,name,location,category
0,#가족여행#관광지#교과서속여행#당일치기여행#봉은사(서울)#사찰&산사,봉은사(서울),강남구,1.0
1,#관광지#근대한옥#기념관#아이와함께#역사#역사문화재#역사속#역사속으로,최순우 옛집,성북구,1.0
2,#15_16한국관광100선#17_18한국관광100선#19_20한국관광100선#가족여행,덕수궁,중구,1.0
3,#관광지,국립민속박물관&국립민속박물관 어린이박물관,종로구,1.0
4,#15_16한국관광100선#17_18한국관광100선#19_20한국관광100선#가족여행,창경궁,종로구,1.0
...,...,...,...,...
1937,#관광지#방이동생태경관보전지역#생태#수도권#자연,방이동 생태경관 보전지역,송파구,8.0
1938,#관광지#자연,한강,성동구,8.0
1939,#관광지#등산#산#수도권#아차산#자연#전망좋은곳,아차산,광진구,8.0
1940,#관광지#자연,반포 서래섬,서초구,8.0


In [39]:
result = result[['name', 'location', 'category', 'tags']]

In [40]:
result

,name,location,category,tags
0,봉은사(서울),강남구,1.0,#가족여행#관광지#교과서속여행#당일치기여행#봉은사(서울)#사찰&산사
1,최순우 옛집,성북구,1.0,#관광지#근대한옥#기념관#아이와함께#역사#역사문화재#역사속#역사속으로
2,덕수궁,중구,1.0,#15_16한국관광100선#17_18한국관광100선#19_20한국관광100선#가족여행
3,국립민속박물관&국립민속박물관 어린이박물관,종로구,1.0,#관광지
4,창경궁,종로구,1.0,#15_16한국관광100선#17_18한국관광100선#19_20한국관광100선#가족여행
...,...,...,...,...
1937,방이동 생태경관 보전지역,송파구,8.0,#관광지#방이동생태경관보전지역#생태#수도권#자연
1938,한강,성동구,8.0,#관광지#자연
1939,아차산,광진구,8.0,#관광지#등산#산#수도권#아차산#자연#전망좋은곳
1940,반포 서래섬,서초구,8.0,#관광지#자연


In [41]:
result.to_csv('./togo.csv', index = False, header=True)